# Avances en el apartado del reconocimiento de voz

### Instalación de la librería

**Instalación de la librería a usar:**

Nota: Usaremos la API de Google Cloud speech-to-text para hacer el reconocimiento de voz.

In [13]:
!pip install --upgrade google-cloud-speech

     -------------------------------------- 273.9/273.9 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-speech
    Found existing installation: google-cloud-speech 2.18.0
    Uninstalling google-cloud-speech-2.18.0:
      Successfully uninstalled google-cloud-speech-2.18.0


In [14]:
from google.cloud import speech
import os

**Creación de la variable de entorno:**

Creamos una variable de entorno que apunta a la ubicación de un archivo Json que contiene la API key proveída por Google cloud.

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/usuario/Jarvis/jarvis-381306-4975f3ce0efb.json"

In [16]:
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:/Users/usuario/Jarvis/jarvis-381306-4975f3ce0efb.json


Para poder usar el reconocimiento de voz de la API debemos crear una variable `client`, una variable `audio` y una variable `config`:

In [17]:
client = speech.SpeechClient()

In [18]:
gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"

In [19]:
audio = speech.RecognitionAudio(uri=gcs_uri)

In [20]:
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code = "en-US"
)

Usamos la función `recognize` para hacer el reconocimiento:

In [21]:
response = client.recognize(config=config, audio=audio)

#Para imprimir más de un resultado
for i in response.results:
    print(i.alternatives[0].transcript)

how old is the Brooklyn Bridge


---

### Reconocimiento de voz de archivos locales

**Reconocimiento de voz de audios cortos:**

Nota: Es posible hacer reconomiento de voz de archivos locales, pero estos archivos tienen una restricción de 60 segs.

Hacemos una función para hacer el reconocimiento de archivos, pasándole la ruta de estos:

In [22]:
def recognize_voice(audio_path):
    from google.cloud import speech
    import io
    
    client = speech.SpeechClient()
    
    with io.open(audio_path, "rb") as audio_file:
        content = audio_file.read()
        
    audio = speech.RecognitionAudio(content=content)
    
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="es-CO"
    )
    
    response = client.recognize(config=config, audio=audio)
    
    return response

Ahora probamos la función apra ver qué tal funciona:

Nota: Los archivos pueden encontrarse en la carpeta Audios.

In [23]:
responses = recognize_voice("C:/Users/usuario/Jarvis/Audios/Grabación2.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

Hola Me llamo Lucas tengo sueño y quiero dormir


In [24]:
responses = recognize_voice("C:/Users/usuario/Jarvis/Audios/Grabación3.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)


Este es mi madre
 y aún hay en el campo
 te amo te amo te amo te amo


In [25]:
responses = recognize_voice("C:/Users/usuario/Jarvis/Audios/Grabación4.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

Esta es una Esta es una prueba de sonido para ver si el modelo reconoce bien las palabras


In [26]:
responses = recognize_voice("C:/Users/usuario/Jarvis/Audios/Prueba IA.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

esto es una pequeña prueba para ver que también hace el modelo la captación de las palabras no sé muy bien que decir por lo que procedería hacer un breve relato de mi día fui a comprar lentes pero estaban muy caros y no quería gastar tanto dinero así que me gasté como 100000 en mi novia XVI


Vemos que la API funciona bien.

---

### Reconocimiento de voz de archivos en la nube

**Reconocimiento de voz de audios largos:**

Nota: Se puede hacer el reconocimiento de voz de audios de duración mayor a 60 segs, pero no se puede hacer con audios locales.

En este caso los audios deben estar en un bucket de google cloud.

Una función para hacer este reconocimiento de voz:

In [27]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print("Transcript: {}".format(result.alternatives[0].transcript))
        print("Confidence: {}".format(result.alternatives[0].confidence))

---

### Reconocimiento de voz de transmisión continua

**Reconocimiento de voz de audios en transmisión:**

Nota: El reconocimiento de audio de transmisión (el que necesitamos) solo está disponible a través de gRCP.

Instalamos e importamos las librerías necesarias:

In [28]:
!pip install pyaudio

In [44]:
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue

Creamos las variables `RATE` y `CHUNK` que nos servirán como parámetros y son las propiedades default del audio:

In [45]:
RATE = 16000
CHUNK = int(RATE / 10)

Ahora creamos la clase `MicrophoneStream` que servirá para hacer la transmisión de audio desde el micrófono conectado al dispositivo:

In [49]:
class MicrophoneStream(object):
     # Opens a recording stream as a generator yielding the audio chunks.
    
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        
        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True
    
    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer
        )
        
        self.closed = False
        
        return self
        
    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        
        self._buff.put(None)
        self._audio_interface.terminate()
        
    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        self._buff.put(in_data)
        return None, pyaudio.paContinue
    
#     Este es el método que hace el reconocimiento de la transmisión y genera las palabras.
    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            
            if chunk is None:
                return 
            data = [chunk]
            
            while True:
                try:
                    chunk = self._buff.get(block=False)
                    
                    if chunk is None:
                        return 
                    data.append(chunk)
                except queue.Empty:
                    break
                    
            yield b"".join(data)
            
    def stop(self):
        self.closed = True
        self._buff.put(None)

En este punto es donde se hacen los llamados a la clase que acabamos de crear para poder inciar la trmansmisión.

In [ ]:
def main():
    language_code = "es-CO"

    client = speech.SpeechClient()

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code=language_code
    )

    streaming_config = speech.StreamingRecognitionConfig(
        config=config,
        interim_results=True
    )
# Se crea el objeto de tipo MicrophoneStream.
    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        
        requests = (
            speech.StreamingRecognizeRequest(audio_content=content) 
            for content in audio_generator
        )

        responses = client.streaming_recognize(streaming_config, requests)

        listen_print_loop(responses)

def listen_print_loop(responses):
    num_chars_printed = 0

    for response in responses:
        if not response.results:
            continue

        result = response.results[0]

        if not result.alternatives:
            continue

        transcript = result.alternatives[0].transcript

        overwrite_chars = " " * (num_chars_printed - len(transcript))

        if not result.is_final:
            sys.stdout.write(transcript + overwrite_chars + "\r")
            sys.stdout.flush()

            num_chars_printed = len(transcript)

        else:
            print(transcript + overwrite_chars)

            if re.search(r"\b(Terminar|terminar)\b", transcript, re.I):
                print("Finalizando transmisión...")
                break

            num_chars_printed = 0

if __name__ == "__main__":
    main()